# Функции — использование встроенных и создание собственных
Для зачета необходимо выполнить первые три задания. Остальные задания - на зачет с отличием, выполняются по желанию.

## Задание 1
Дан список вида:
data = [
[13, 25, 23, 34],
[45, 32, 44, 47],
[12, 33, 23, 95],
[13, 53, 34, 35],
]

Напишите функцию, которая возвращает сумму элементов на диагонали. Т. е. 13+32+23+35.

In [1]:
data = [[13, 25, 23, 34],[45, 32, 44, 47],[12, 33, 23, 95],[13, 53, 34, 35],]

In [2]:
def diag_summ(ar):
    return sum([ar[i][i] for i in range(len(ar))])

In [3]:
diag_summ(data), 13+32+23+35

(103, 103)

## Задание 2
Дан список чисел, часть из которых имеют строковый тип или содержат буквы. Напишите функцию, которая возвращает сумму квадратов элементов, которые могут быть числами.
data = [1, ‘5’, ‘abc’, 20, ‘2’]

In [4]:
data = [1, '5', 'abc', 20, '2']

In [5]:
def int_sum_square(ar):
    summ = 0
    for x in ar:
        try:
            summ += int(x)**2
        except:
            pass
    return summ

In [6]:
int_sum_square(data), 1+5**2+20**2+2**2

(430, 430)

## Задание 3
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [7]:
import requests
import numpy as np

In [8]:
class Rate:
    def __init__(self, format='value'):
        self.format = format
    
    @staticmethod
    def exchange_rates():
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    @classmethod
    def get_max_price_crncy(cls):
        rates = list(cls.exchange_rates().values())
        max_ind = np.argmax([d['Value'] for d in rates])
        return [d['Name'] for d in rates][max_ind]
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')

In [9]:
r = Rate()
r.get_max_price_crncy()

'Датских крон'

## Задание 4
Последнее упражнение с занятия

Добавьте в класс еще один формат, который возвращает название валюты (например, ‘Евро’).

Добавьте в класс параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением.

In [10]:
class Rate:
    def __init__(self, diff, format='value'):
        self.format = format
        self.diff = diff
    
    @staticmethod
    def exchange_rates():
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency, _format=None):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        _format = _format if _format else self.format
        
        response = self.exchange_rates()
        
        if currency in response:
            if _format == 'full':
                return response[currency]
            
            if _format == 'value':
                return response[currency]['Value']
            
            if _format == 'name':
                return response[currency]['Name']
            
            if _format == 'previous':
                return response[currency]['Previous']
        
        return 'Error'
    
    @classmethod
    def get_max_price_crncy(cls):
        rates = list(cls.exchange_rates().values())
        max_ind = np.argmax([d['Value'] for d in rates])
        return [d['Name'] for d in rates][max_ind]
    
    def get_crncy(self, crncy):
        if self.diff:
            return self.make_format(crncy, 'value')-self.make_format(crncy, 'previous')
        return self.make_format(crncy)
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.get_crncy('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.get_crncy('USD')

In [11]:
r = Rate(format='value', diff=True)
r.eur()

-0.2834000000000003

## Задание 5
Напишите функцию, возвращающую сумму первых n чисел Фибоначчи

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377

In [12]:
def Fib(n):
    phi = (1+5**0.5)/2
    return int(np.round(phi**n/(5**0.5)))

def sum_fib(n):
    return Fib(n+2)-1  

In [13]:
def Fib_rec(n):
    if n == 0 or n == 1:
        return n
    return Fib_rec(n-1)+Fib_rec(n-2)    

def sum_fib_rec(n):
    return sum([Fib_rec(x) for x in range(n+1)])

In [14]:
%timeit sum_fib(10)
%timeit sum_fib_rec(10)

4.65 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
50.7 µs ± 5.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Задание 6
Напишите функцию, преобразующую произвольный список вида [‘2018-01-01’, ‘yandex’, ‘cpc’, 100] в словарь {‘2018-01-01’: {‘yandex’: {‘cpc’: 100}}}

In [15]:
lst = ['2018-01-01', 'yandex','cpc',100]

In [16]:
def lst_to_dict(lst):
    dct = {lst[-2]: lst[-1]}
    for x in lst[-3::-1]:
        dct = {x:dct}
    return dct

In [17]:
lst_to_dict(lst)

{'2018-01-01': {'yandex': {'cpc': 100}}}